# 크롤링 실습

## 크롤링 실습을 위한 준비

In [1]:
!pip3 install requests
!pip3 install bs4
!pip3 install selenium

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=d5daff1a46e93b503d0d4867bb78894abc24c5aa0ac42559d736257908e2b25b
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


## GET 요청하기

In [2]:
import requests

url = 'https://search.naver.com/search.naver?ie=UTF-8&query=%EC%A0%9C%EC%A3%BC%EB%8F%84&sm=chr_hty'
res = requests.get(url)

print(res) # 응답 객체
# res.text # HTML 텍스트

<Response [200]>


#### HTTP 상태 코드
 - 1xx (정보): 요청을 받았으며 프로세스를 계속한다
 - 2xx (성공): 요청을 성공적으로 받았으며 인식했고 수용하였다
 - 3xx (리다이렉션): 요청 완료를 위해 추가 작업 조치가 필요하다
 - 4xx (클라이언트 오류): 요청의 문법이 잘못되었거나 요청을 처리할 수 없다
 - 5xx (서버 오류): 서버가 명백히 유효한 요청에 대해 충족을 실패했다

[출처: 위키피디아](https://ko.wikipedia.org/wiki/HTTP_%EC%83%81%ED%83%9C_%EC%BD%94%EB%93%9C)

In [3]:
res.status_code # HTTP 상태 코드

200

## BeautifulSoup을 이용한 크롤링

In [4]:
# 임시 html 텍스트
html = '''
<html>
  <head>
    <title>Example HTML</title>
  </head>
  <body>
    <div id='first' class='Example'>
      <h3 title='Good Content Title'>Contents Title</h3>
      <p>Test contents</p>
    </div>
    <div id='second' class='Example'>
      <p>Text1</p>
      <p>Text2</p>
      <p>Text3</p>
    </div>
  </body>
</html>
'''

In [5]:
from bs4 import BeautifulSoup

# 첫 번째 인자: 파싱할 객체
# 두 번째 인자: 사용할 parser(구문 분석기)
soup = BeautifulSoup(html, 'html.parser')

find(), find_all()

태그 검색   
- find(tag, attributes, recursive, text, keywords)   
- find_all(tag, attributes, recursive, text, limit, keywords)   

get_text()

태그 내 텍스트 반환(부모 태그의 경우 자식 태그의 텍스트를 모두 반환)

In [6]:
tag = soup.find('h3')
print(tag)
tag.get_text()

<h3 title="Good Content Title">Contents Title</h3>


'Contents Title'

In [ ]:
tag = soup.find('p')
print(tag)
tag.get_text()

<p>Test contents</p>


'Test contents'

In [7]:
# find_all은 list 형태로 값 반환
soup.find_all('p')

[<p>Test contents</p>, <p>Text1</p>, <p>Text2</p>, <p>Text3</p>]

In [8]:
soup.find_all('p')[2].get_text()

'Text2'

In [9]:
# 개수 제한
soup.find_all('p', limit=2)

[<p>Test contents</p>, <p>Text1</p>]

In [10]:
# 여러 태그
soup.find_all({'p', 'h3'})

[<h3 title="Good Content Title">Contents Title</h3>,
 <p>Test contents</p>,
 <p>Text1</p>,
 <p>Text2</p>,
 <p>Text3</p>]

In [ ]:
soup.find('div')

<div class="Example" id="first">
<h3 title="Good Content Title">Contents Title</h3>
<p>Test contents</p>
</div>

In [ ]:
soup.find('div').get_text().replace('\n', '')

'Contents TitleTest contents'

In [ ]:
# 특정 속성을 가진 태그 검색
soup.find('div', {'id': 'second'})

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

In [ ]:
# keyword 매개변수 이용
soup.find('div', id='second')

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

In [ ]:
soup.find('div', id='second', class_='Example')

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

In [ ]:
# 속성이 여러 개일 경우
attrs = {
    'id': 'second',
    'class': 'Example'
}
soup.find('div', attrs)

<div class="Example" id="second">
<p>Text1</p>
<p>Text2</p>
<p>Text3</p>
</div>

네이버 금융 주요뉴스 헤드라인 크롤링 예시

In [11]:
url = "https://finance.naver.com/"
res = requests.get(url)
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser') # 문서 구조에 따라 'html.parser', 'lxml', 'xml' 등 다양한 구조 분석 방식을 사용할 수 있음

In [ ]:
url = "https://finance.naver.com/"
res = requests.get(url)

# 첫번째 인자를 텍스트로 넣어줘야 함
soup = BeautifulSoup(res.text, 'html.parser') # 문서 구조에 따라 'html.parser', 'lxml', 'xml' 등 다양한 구조 분석 방식을 사용할 수 있음

# 주요뉴스 섹션 검색
strategy = soup.find('div', class_="section_strategy")

# 주요뉴스 헤드라인을 저장할 리스트
strategies = []

# 주요뉴스에 있는 헤드라인 텍스트 정보 추출한 뒤 리스트에 저장
for i in range(len(strategy.find_all('a'))-1):
  strategies.append(strategy.find_all('a')[i].get_text())

# 결과 확인
for headline in strategies:
  print(headline)


1월 가상자산 큰 장 선다…JP모건도 슬쩍 참전
남양유업-한앤코 소송전 운명의 날...홍원식 앞에 쌓인 ‘청구서’[마켓인]
태영건설 사태에도…한화에어로 올해 첫 수요예측 흥행[마켓인]
진격의 방산株…하락장 뚫고 고공행진
그동안 너무 많이 올랐나…'8만전자' 눈앞서 꺾였다
강석훈 산은 회장 "태영 약속 안지켜. 열심하겠으니 도와달라는 격"


# Selenium을 활용한 크롤링 예시:네이버 토요일, 일요일 웹툰 정보 크롤링
- 로컬 환경에서 진행

### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

# Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!